In [1]:
from datasets import load_dataset

ds = load_dataset("akariasai/PopQA")
df = ds["test"].to_pandas()


/Users/kdu/micromamba/envs/assertions/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Repo card metadata block was not found. Setting CardData to empty.


In [2]:
# Sample 10 rows from each prop and display subj, obj, and prop
for prop in df["prop"].unique():
    print(f"\n=== {prop} ===")
    prop_subset = df[df["prop"] == prop]
    sample_size = min(10, len(prop_subset))
    sampled_rows = prop_subset.sample(n=sample_size, random_state=42)
    print(sampled_rows[["subj", "obj", "prop"]].to_string(index=False))



=== occupation ===
                subj            obj       prop
       Scooter Braun talent manager occupation
   Virginia Haviland      librarian occupation
       Annie Beustes     politician occupation
          Ali Hillis          actor occupation
   Guy Joseph Bonnet  major general occupation
      Eliza Lawrence     politician occupation
         Alan Huckle       diplomat occupation
Charles Martin Smith          actor occupation
      Jörg A. Eggers   screenwriter occupation
          Jon Eikemo          actor occupation

=== place of birth ===
              subj          obj           prop
      Paul Braniff   Portaferry place of birth
Alexander Butlerov       Çistay place of birth
      Mike Gregory         Bath place of birth
 Christian Vaquero   Montevideo place of birth
 Alexandros Matsas       Athens place of birth
    Charles Plumer  Canons Park place of birth
       Bob Warlick      Hickory place of birth
       Terry Hunte Saint Philip place of birth
       James Bro

In [3]:
# filter out capital of because that one is weird
df = df[df["prop"] != "capital of"]
df

,id,subj,prop,obj,subj_id,prop_id,obj_id,s_aliases,o_aliases,s_uri,o_uri,s_wiki_title,o_wiki_title,s_pop,o_pop,question,possible_answers
0,4222362,George Rankin,occupation,politician,1850297,22,2834605,"[""George James Rankin""]","[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q5543720,http://www.wikidata.org/entity/Q82955,George Rankin,Politician,142,25692,What is George Rankin's occupation?,"[""politician"", ""political leader"", ""political ..."
1,4725190,John Mayne,occupation,journalist,2079053,22,663400,[],"[""journo"",""journalists""]",http://www.wikidata.org/entity/Q6247345,http://www.wikidata.org/entity/Q1930187,John Mayne,Journalist,236,24952,What is John Mayne's occupation?,"[""journalist"", ""journo"", ""journalists""]"
2,4382392,Henry Feilden,occupation,politician,1925450,22,2834605,"[""Henry Master Feilden""]","[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q5725578,http://www.wikidata.org/entity/Q82955,Henry Feilden (Conservative politician),Politician,58,25692,What is Henry Feilden's occupation?,"[""politician"", ""political leader"", ""political ..."
3,4822110,Kathy Saltzman,occupation,politician,2122743,22,2834605,[],"[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q6377295,http://www.wikidata.org/entity/Q82955,Kathy Saltzman,Politician,127,25692,What is Kathy Saltzman's occupation?,"[""politician"", ""political leader"", ""political ..."
4,4011112,Eleanor Davis,occupation,cartoonist,1752619,22,68412,"[""Eleanor McCutcheon Davis""]","[""graphic artist"",""animator"",""illustrator""]",http://www.wikidata.org/entity/Q5354261,http://www.wikidata.org/entity/Q1114448,Eleanor Davis,Cartoonist,317,9649,What is Eleanor Davis's occupation?,"[""cartoonist"", ""graphic artist"", ""animator"", ""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14262,4165948,Netherlands,capital,Amsterdam,1825504,422,2455815,"[""Holland"",""the Netherlands"",""NL"",""NED"",""Neder...","[""Mokum"",""Amsterdam, NL"",""Amsterdam, Netherlan...",http://www.wikidata.org/entity/Q55,http://www.wikidata.org/entity/Q727,Netherlands,Amsterdam,416350,147710,What is the capital of Netherlands?,"[""The Hague"", ""Den Haag"", ""'s-Gravenhage"", ""Ha..."
14263,2617063,City Municipality of Celje,capital,Celje,1127153,422,5035,"[""Celje city municipality""]",[],http://www.wikidata.org/entity/Q3441823,http://www.wikidata.org/entity/Q1012,City Municipality of Celje,Celje,335,4086,What is the capital of City Municipality of Ce...,"[""Celje""]"
14264,471864,Delhi,capital,New Delhi,192624,422,2938512,"[""National Capital Region of Delhi"",""NCR""]","[""New Delhi district"",""Nayi Dilli""]",http://www.wikidata.org/entity/Q1353,http://www.wikidata.org/entity/Q987,Delhi,New Delhi,219227,128193,What is the capital of Delhi?,"[""New Delhi"", ""New Delhi district"", ""Nayi Dilli""]"
14265,2965573,Palestinian territories,capital,Jerusalem,1270077,422,127625,"[""occupied Palestinian territories"",""OPT"",""Wes...","[""Yerushalayim"",""J'lem"",""Aelia Capitolina"",""Al...",http://www.wikidata.org/entity/Q407199,http://www.wikidata.org/entity/Q1218,Palestinian territories,Jerusalem,17545,136734,What is the capital of Palestinian territories?,"[""Jerusalem"", ""Yerushalayim"", ""J'lem"", ""Aelia ..."


In [4]:
print(df["prop"].value_counts())
print(df[df["subj"] == "Pilot"].head())
print(df[["subj", "prop"]].value_counts())

prop
director          1999
screenwriter      1999
genre             1619
producer          1520
author            1514
composer           978
country            838
capital            645
place of birth     584
father             570
sport              547
occupation         532
religion           338
mother             187
color               34
Name: count, dtype: int64
           id   subj      prop               obj  subj_id  prop_id   obj_id  \
5763  5453595  Pilot  director  Francis Lawrence  2425271      526  1883124   
5805  5453492  Pilot  director        Peter Berg  2425233      526   607993   
5818  4577696  Pilot  director     Marc Buckland  2018998      526  1230313   
5932  2957316  Pilot  director     Phillip Noyce  1266576      526  1791491   
5957   149939  Pilot  director      Brett Ratner    60507      526  1052983   

                        s_aliases o_aliases  \
5763  ["Tales of the Red Thread"]        []   
5805                           []        []   
5818    

In [5]:
# Filter out rows where the combination of subj and prop appears more than once
subj_prop_counts = df.groupby(["subj", "prop"]).size()
duplicated_pairs = subj_prop_counts[subj_prop_counts > 1].index
df_filtered = df.set_index(["subj", "prop"]).drop(duplicated_pairs).reset_index()
df = df_filtered


In [6]:
df[["subj", "prop"]].value_counts()


subj                 prop      
"Them"               producer      1
Ruth Berman Harris   genre         1
Ruslan and Lyudmila  composer      1
Russell Stokes       occupation    1
Russia               capital       1
                                  ..
Hara                 genre         1
Harald Molander      mother        1
Harbor Lights        composer      1
Hard                 producer      1
∞                    producer      1
Name: count, Length: 11981, dtype: int64

In [7]:
df[df["subj"] == "London"]

,subj,prop,id,obj,subj_id,prop_id,obj_id,s_aliases,o_aliases,s_uri,o_uri,s_wiki_title,o_wiki_title,s_pop,o_pop,question,possible_answers
1991,London,genre,585660,punk rock,239634,91,1011970,[],"[""punk"",""punk music""]",http://www.wikidata.org/entity/Q1442133,http://www.wikidata.org/entity/Q3071,London (punk band),Punk rock,534,83910,What genre is London?,"[""punk rock"", ""punk"", ""punk music""]"
6262,London,director,5010594,Sundar C.,2212451,526,2619857,[],"[""Vinayagar Sundar Vel""]",http://www.wikidata.org/entity/Q6669761,http://www.wikidata.org/entity/Q7639214,London (2005 Indian film),Sundar C.,1864,58073,Who was the director of London?,"[""Sundar C."", ""Vinayagar Sundar Vel""]"
10755,London,author,5010599,William Blake,2212454,484,1279797,[],"[""W. Blake"",""Uil'iam Bleik"",""Blake""]",http://www.wikidata.org/entity/Q6669771,http://www.wikidata.org/entity/Q41513,London (William Blake poem),William Blake,4792,78458,Who is the author of London?,"[""William Blake"", ""W. Blake"", ""Uil'iam Bleik"",..."


In [8]:
df[["obj", "prop"]].value_counts()


obj                       prop        
association football      sport           327
United States of America  country         261
politician                occupation      139
Catholic Church           religion        134
actor                     occupation       98
                                         ... 
Howard Koch               screenwriter      1
Howard Hawks              screenwriter      1
Howard Goodall            composer          1
Howard Estabrook          screenwriter      1
Ümit Ünal                 screenwriter      1
Name: count, Length: 7214, dtype: int64

In [9]:
df[df["obj"] == "association football"]

,subj,prop,id,obj,subj_id,prop_id,obj_id,s_aliases,o_aliases,s_uri,o_uri,s_wiki_title,o_wiki_title,s_pop,o_pop,question,possible_answers
9321,2012 Georgetown Hoyas men's soccer team,sport,3175769,association football,1358727,560,920845,[],"[""football"",""soccer""]",http://www.wikidata.org/entity/Q4625659,http://www.wikidata.org/entity/Q2736,2012 Georgetown Hoyas men's soccer team,Association football,46,197767,What sport does 2012 Georgetown Hoyas men's so...,"[""association football"", ""football"", ""soccer""]"
9323,2014–15 Premier League of Bosnia and Herzegovina,sport,1088874,association football,470931,560,920845,"[""2014-15 Premier League of Bosnia and Herzego...","[""football"",""soccer""]",http://www.wikidata.org/entity/Q16875416,http://www.wikidata.org/entity/Q2736,2014–15 Premier League of Bosnia and Herzegovina,Association football,213,197767,What sport does 2014–15 Premier League of Bosn...,"[""association football"", ""football"", ""soccer""]"
9324,2008–09 Maltese Premier League,sport,1447026,association football,646974,560,920845,"[""2008-09 Maltese Premier League""]","[""football"",""soccer""]",http://www.wikidata.org/entity/Q1887835,http://www.wikidata.org/entity/Q2736,2008–09 Maltese Premier League,Association football,136,197767,What sport does 2008–09 Maltese Premier League...,"[""association football"", ""football"", ""soccer""]"
9326,Shaun Carey,sport,5738225,association football,2561318,560,920845,"[""Shaun Peter Carey""]","[""football"",""soccer""]",http://www.wikidata.org/entity/Q7490810,http://www.wikidata.org/entity/Q2736,Shaun Carey,Association football,124,197767,What sport does Shaun Carey play?,"[""association football"", ""football"", ""soccer""]"
9328,Mustafa Keçeli,sport,1519072,association football,678095,560,920845,[],"[""football"",""soccer""]",http://www.wikidata.org/entity/Q1955557,http://www.wikidata.org/entity/Q2736,Mustafa Keçeli,Association football,104,197767,What sport does Mustafa Keçeli play?,"[""association football"", ""football"", ""soccer""]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9861,Marc Herold Gracien,sport,2871635,association football,1230317,560,920845,"[""Marc H\u00e9rold Gracien""]","[""football"",""soccer""]",http://www.wikidata.org/entity/Q3845367,http://www.wikidata.org/entity/Q2736,Marc Hérold Gracien,Association football,83,197767,What sport does Marc Herold Gracien play?,"[""association football"", ""football"", ""soccer""]"
9863,Barrie Aitchison,sport,80903,association football,31448,560,920845,"[""Barrie George Aitchison""]","[""football"",""soccer""]",http://www.wikidata.org/entity/Q10555078,http://www.wikidata.org/entity/Q2736,Barrie Aitchison,Association football,82,197767,What sport does Barrie Aitchison play?,"[""association football"", ""football"", ""soccer""]"
9864,Leyton Orient F.C.,sport,1528899,association football,682813,560,920845,"[""Leyton Orient Football Club"",""Leyton Orient ...","[""football"",""soccer""]",http://www.wikidata.org/entity/Q19595,http://www.wikidata.org/entity/Q2736,Leyton Orient F.C.,Association football,25844,197767,What sport does Leyton Orient F.C. play?,"[""association football"", ""football"", ""soccer""]"
9865,Jean,sport,3676450,association football,1596825,560,920845,"[""Jean Raphael Vanderlei Moreira""]","[""football"",""soccer""]",http://www.wikidata.org/entity/Q506861,http://www.wikidata.org/entity/Q2736,"Jean (footballer, born 1986)",Association football,316,197767,What sport does Jean play?,"[""association football"", ""football"", ""soccer""]"


In [10]:
df.columns

Index(['subj', 'prop', 'id', 'obj', 'subj_id', 'prop_id', 'obj_id',
       's_aliases', 'o_aliases', 's_uri', 'o_uri', 's_wiki_title',
       'o_wiki_title', 's_pop', 'o_pop', 'question', 'possible_answers'],
      dtype='object')

In [11]:
def is_bijective(df, col_a, col_b):
    # Check if A->B mapping is one-to-one
    a_to_b_unique = df.groupby(col_a)[col_b].nunique().max() == 1
    
    # Check if B->A mapping is one-to-one
    b_to_a_unique = df.groupby(col_b)[col_a].nunique().max() == 1
    
    # Both conditions must be true
    return a_to_b_unique and b_to_a_unique

# Example usage
result = is_bijective(df, 'o_uri', 'obj_id')
print(f"Columns are bijective: {result}")

result = is_bijective(df, 'o_aliases', 'possible_answers')
print(f"Columns are bijective: {result}")

Columns are bijective: True
Columns are bijective: False


In [12]:
# Filter out all obj_ids that have only 1 row
# Keep only obj_ids that appear more than once

# Count how many times each obj_id appears
obj_id_counts = df['obj_id'].value_counts()

# Get obj_ids that appear more than once
obj_ids_multiple = obj_id_counts[obj_id_counts > 1].index

# Filter the dataframe to keep only rows with obj_ids that appear multiple times
df_filtered = df[df['obj_id'].isin(obj_ids_multiple)]

print(f"Original dataframe shape: {df.shape}")
print(f"Filtered dataframe shape: {df_filtered.shape}")
print(f"Removed {df.shape[0] - df_filtered.shape[0]} rows with unique obj_ids")

# Show some statistics
print(f"\nOriginal obj_id counts:")
print(f"  Total unique obj_ids: {df['obj_id'].nunique()}")
print(f"  obj_ids with count = 1: {(obj_id_counts == 1).sum()}")
print(f"  obj_ids with count > 1: {(obj_id_counts > 1).sum()}")

print(f"\nFiltered obj_id counts:")
filtered_counts = df_filtered['obj_id'].value_counts()
print(f"  Total unique obj_ids: {df_filtered['obj_id'].nunique()}")
print(f"  Min count per obj_id: {filtered_counts.min()}")
print(f"  Max count per obj_id: {filtered_counts.max()}")

# Update the main dataframe
df = df_filtered


Original dataframe shape: (11984, 17)
Filtered dataframe shape: (7195, 17)
Removed 4789 rows with unique obj_ids

Original obj_id counts:
  Total unique obj_ids: 6402
  obj_ids with count = 1: 4789
  obj_ids with count > 1: 1613

Filtered obj_id counts:
  Total unique obj_ids: 1613
  Min count per obj_id: 2
  Max count per obj_id: 327


In [13]:
# Find rows where o_aliases and possible_answers are not bijective

print("=== ANALYZING NON-BIJECTIVE RELATIONSHIPS ===\n")

# Check if columns exist
if 'o_aliases' not in df.columns:
    print("Error: 'o_aliases' column not found in dataframe")
    print("Available columns:", df.columns.tolist())
elif 'possible_answers' not in df.columns:
    print("Error: 'possible_answers' column not found in dataframe") 
    print("Available columns:", df.columns.tolist())
else:
    # First, let's check overall bijection status
    result = is_bijective(df, 'o_aliases', 'possible_answers')
    print(f"Overall bijection status: {result}")
    
    if not result:
        print("\n=== FINDING BIJECTION VIOLATIONS ===\n")
        
        # Convert to string representation for grouping (since these might be lists)
        df_str = df.copy()
        df_str['o_aliases_str'] = df_str['o_aliases'].astype(str)
        df_str['possible_answers_str'] = df_str['possible_answers'].astype(str)
        
        # Check o_aliases -> possible_answers violations (one o_aliases maps to multiple possible_answers)
        alias_to_answers = df_str.groupby('o_aliases_str')['possible_answers_str'].nunique()
        alias_violations = alias_to_answers[alias_to_answers > 1]
        
        if len(alias_violations) > 0:
            print(f"1. ALIASES -> ANSWERS VIOLATIONS: {len(alias_violations)} aliases map to multiple answer sets")
            print("   (Same alias appears with different possible_answers)\n")
            
            for alias_str in alias_violations.index[:5]:  # Show first 5
                matching_rows = df_str[df_str['o_aliases_str'] == alias_str]
                print(f"   Alias: {matching_rows.iloc[0]['o_aliases']}")
                print(f"   Maps to {len(matching_rows)} different answer sets:")
                for i, (_, row) in enumerate(matching_rows.iterrows()):
                    print(f"     {i+1}. {row['possible_answers']} (subj: {row['subj']}, prop: {row['prop']})")
                print()
        
        # Check possible_answers -> o_aliases violations (one possible_answers maps to multiple o_aliases)
        answers_to_alias = df_str.groupby('possible_answers_str')['o_aliases_str'].nunique()
        answer_violations = answers_to_alias[answers_to_alias > 1]
        
        if len(answer_violations) > 0:
            print(f"2. ANSWERS -> ALIASES VIOLATIONS: {len(answer_violations)} answer sets map to multiple aliases")
            print("   (Same possible_answers appears with different o_aliases)\n")
            
            for answers_str in answer_violations.index[:5]:  # Show first 5
                matching_rows = df_str[df_str['possible_answers_str'] == answers_str]
                print(f"   Possible answers: {matching_rows.iloc[0]['possible_answers']}")
                print(f"   Maps to {len(matching_rows)} different aliases:")
                for i, (_, row) in enumerate(matching_rows.iterrows()):
                    print(f"     {i+1}. {row['o_aliases']} (subj: {row['subj']}, prop: {row['prop']})")
                print()
        
        # Show some sample problematic rows
        print("=== SAMPLE PROBLEMATIC ROWS ===\n")
        
        if len(alias_violations) > 0:
            sample_alias = alias_violations.index[0]
            problem_rows = df_str[df_str['o_aliases_str'] == sample_alias]
            print("Sample rows with same o_aliases but different possible_answers:")
            print(problem_rows[['subj', 'obj', 'prop', 'o_aliases', 'possible_answers']].to_string())
            print()
        
        if len(answer_violations) > 0:
            sample_answer = answer_violations.index[0]
            problem_rows = df_str[df_str['possible_answers_str'] == sample_answer]
            print("Sample rows with same possible_answers but different o_aliases:")
            print(problem_rows[['subj', 'obj', 'prop', 'o_aliases', 'possible_answers']].to_string())
    
    else:
        print("The columns are bijective - no violations found!")


=== ANALYZING NON-BIJECTIVE RELATIONSHIPS ===

Overall bijection status: False

=== FINDING BIJECTION VIOLATIONS ===

1. ALIASES -> ANSWERS VIOLATIONS: 147 aliases map to multiple answer sets
   (Same alias appears with different possible_answers)

   Alias: ["Abraham Stoker"]
   Maps to 2 different answer sets:
     1. ["Bram Stoker", "Abraham Stoker", "Jimmy Sangster", "James Henry Kinmel Sangster"] (subj: Dracula, prop: screenwriter)
     2. ["Bram Stoker", "Abraham Stoker"] (subj: The Man, prop: author)

   Alias: ["Agatha Mary Clarissa Christie","Agatha Mary Clarissa Miller","Mary Westmacott","Agatha Mary Clarissa Mallowan"]
   Maps to 5 different answer sets:
     1. ["Agatha Christie", "Agatha Mary Clarissa Christie", "Agatha Mary Clarissa Miller", "Mary Westmacott", "Agatha Mary Clarissa Mallowan", "Anthony Shaffer", "Anthony Joshua Shaffer"] (subj: Death on the Nile, prop: screenwriter)
     2. ["Agatha Christie", "Agatha Mary Clarissa Christie", "Agatha Mary Clarissa Miller",

In [14]:
# Get other facts related to the object 
groups = df.groupby("obj_id")[["subj", "prop", "obj"]]
for _, group in groups:
    print(f"OBJECT: {group.iloc[0]['obj']}")
    print(group)
    print("\n\n")

OBJECT: Boston
                subj            prop     obj
671    Donald De Lue  place of birth  Boston
758       Walter Gay  place of birth  Boston
11405  Massachusetts         capital  Boston



OBJECT: Salvatore Samperi
                   subj          prop                obj
7583  Come Play with Me  screenwriter  Salvatore Samperi
7657          Malicious  screenwriter  Salvatore Samperi
7992         Submission  screenwriter  Salvatore Samperi



OBJECT: Carlo Vanzina
                   subj          prop            obj
7132         The Gamble  screenwriter  Carlo Vanzina
7512  Vacanze di Natale  screenwriter  Carlo Vanzina



OBJECT: Celje
                             subj            prop    obj
961                   Alma Karlin  place of birth  Celje
11980  City Municipality of Celje         capital  Celje



OBJECT: Liberia
                   subj     prop      obj
3371  River Cess County  country  Liberia
3875       Hotel Africa  country  Liberia



OBJECT: Buronson
         su

In [15]:
import pandas as pd
import json
import numpy as np
import random
from ast import literal_eval
import json
random.seed(42)
np.random.seed(42)

df.to_csv("popqa_filtered.csv", index=False)
# df = pd.read_csv("popqa_filtered.csv", converters={"o_aliases": ast.literal_eval, "possible_answers": ast.literal_eval, "s_aliases": ast.literal_eval}, encoding='utf-8')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7195 entries, 0 to 11982
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   subj              7195 non-null   object
 1   prop              7195 non-null   object
 2   id                7195 non-null   int64 
 3   obj               7195 non-null   object
 4   subj_id           7195 non-null   int64 
 5   prop_id           7195 non-null   int64 
 6   obj_id            7195 non-null   int64 
 7   s_aliases         7195 non-null   object
 8   o_aliases         7195 non-null   object
 9   s_uri             7195 non-null   object
 10  o_uri             7195 non-null   object
 11  s_wiki_title      7195 non-null   object
 12  o_wiki_title      7195 non-null   object
 13  s_pop             7195 non-null   int64 
 14  o_pop             7195 non-null   int64 
 15  question          7195 non-null   object
 16  possible_answers  7195 non-null   object
dtypes: int64(6), objec

In [17]:
df.iloc[0]["possible_answers"]

'["politician", "political leader", "political figure", "polit.", "pol"]'

In [18]:
df[df["obj"] == "politician"]

,subj,prop,id,obj,subj_id,prop_id,obj_id,s_aliases,o_aliases,s_uri,o_uri,s_wiki_title,o_wiki_title,s_pop,o_pop,question,possible_answers
0,George Rankin,occupation,4222362,politician,1850297,22,2834605,"[""George James Rankin""]","[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q5543720,http://www.wikidata.org/entity/Q82955,George Rankin,Politician,142,25692,What is George Rankin's occupation?,"[""politician"", ""political leader"", ""political ..."
2,Henry Feilden,occupation,4382392,politician,1925450,22,2834605,"[""Henry Master Feilden""]","[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q5725578,http://www.wikidata.org/entity/Q82955,Henry Feilden (Conservative politician),Politician,58,25692,What is Henry Feilden's occupation?,"[""politician"", ""political leader"", ""political ..."
3,Kathy Saltzman,occupation,4822110,politician,2122743,22,2834605,[],"[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q6377295,http://www.wikidata.org/entity/Q82955,Kathy Saltzman,Politician,127,25692,What is Kathy Saltzman's occupation?,"[""politician"", ""political leader"", ""political ..."
7,Leona Detiège,occupation,1758574,politician,777898,22,2834605,"[""Leona Detiege""]","[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q2261198,http://www.wikidata.org/entity/Q82955,Leona Detiège,Politician,124,25692,What is Leona Detiège's occupation?,"[""politician"", ""political leader"", ""political ..."
8,"William Murray, 1st Earl of Mansfield",occupation,6339290,politician,2858390,22,2834605,"[""William Murray, Earl of Mansfield""]","[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q869340,http://www.wikidata.org/entity/Q82955,"William Murray, 1st Earl of Mansfield",Politician,5766,25692,"What is William Murray, 1st Earl of Mansfield'...","[""politician"", ""political leader"", ""political ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,Nina Varlamova,occupation,5294096,politician,2349303,22,2834605,[],"[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q7038152,http://www.wikidata.org/entity/Q82955,Nina Varlamova,Politician,121,25692,What is Nina Varlamova's occupation?,"[""politician"", ""political leader"", ""political ..."
517,Lorena Corona Valdés,occupation,1419959,politician,635038,22,2834605,[],"[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q18685511,http://www.wikidata.org/entity/Q82955,Lorena Corona Valdés,Politician,35,25692,What is Lorena Corona Valdés's occupation?,"[""politician"", ""political leader"", ""political ..."
520,"Sir Thomas Clarges, 3rd Baronet",occupation,1204905,politician,536660,22,2834605,"[""Thomas Clarges"",""Sir Thomas Clarges""]","[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q17279218,http://www.wikidata.org/entity/Q82955,"Sir Thomas Clarges, 3rd Baronet",Politician,39,25692,"What is Sir Thomas Clarges, 3rd Baronet's occu...","[""politician"", ""political leader"", ""political ..."
521,Donald Robert Macgregor,occupation,3939174,politician,1720098,22,2834605,[],"[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q5295071,http://www.wikidata.org/entity/Q82955,Donald Robert Macgregor,Politician,29,25692,What is Donald Robert Macgregor's occupation?,"[""politician"", ""political leader"", ""political ..."


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7195 entries, 0 to 11982
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   subj              7195 non-null   object
 1   prop              7195 non-null   object
 2   id                7195 non-null   int64 
 3   obj               7195 non-null   object
 4   subj_id           7195 non-null   int64 
 5   prop_id           7195 non-null   int64 
 6   obj_id            7195 non-null   int64 
 7   s_aliases         7195 non-null   object
 8   o_aliases         7195 non-null   object
 9   s_uri             7195 non-null   object
 10  o_uri             7195 non-null   object
 11  s_wiki_title      7195 non-null   object
 12  o_wiki_title      7195 non-null   object
 13  s_pop             7195 non-null   int64 
 14  o_pop             7195 non-null   int64 
 15  question          7195 non-null   object
 16  possible_answers  7195 non-null   object
dtypes: int64(6), objec

In [20]:
df.sort_values(by="s_pop")

,subj,prop,id,obj,subj_id,prop_id,obj_id,s_aliases,o_aliases,s_uri,o_uri,s_wiki_title,o_wiki_title,s_pop,o_pop,question,possible_answers
3238,Valea Seacă River,country,6089822,Romania,2739911,182,756886,[],"[""Roumania"",""Rumania"",""Rom\u00e2nia"",""ro"",""\ud...",http://www.wikidata.org/entity/Q7910319,http://www.wikidata.org/entity/Q218,Valea Seacă River (Mara),Romania,2,278539,In what country is Valea Seacă River?,"[""Romania"", ""Roumania"", ""Rumania"", ""România"", ..."
3864,Valea Pleșii River,country,6089701,Romania,2739797,182,756886,[],"[""Roumania"",""Rumania"",""Rom\u00e2nia"",""ro"",""\ud...",http://www.wikidata.org/entity/Q7910169,http://www.wikidata.org/entity/Q218,Valea Pleșii River (Bârsa),Romania,3,278539,In what country is Valea Pleșii River?,"[""Romania"", ""Roumania"", ""Rumania"", ""România"", ..."
9301,Ecclesiastical Statistics,religion,3979417,Catholic Church,1739573,106,2918816,[],"[""Roman Catholic Church"",""Church"",""Roman Apost...",http://www.wikidata.org/entity/Q5332261,http://www.wikidata.org/entity/Q9592,Ecclesiastical statistics,Catholic Church,3,195995,What is the religion of Ecclesiastical Statist...,"[""Catholic Church"", ""Roman Catholic Church"", ""..."
3365,Fântâneaua Rece River,country,4176872,Romania,1831227,182,756886,"[""Fantaneaua Rece River""]","[""Roumania"",""Rumania"",""Rom\u00e2nia"",""ro"",""\ud...",http://www.wikidata.org/entity/Q5511438,http://www.wikidata.org/entity/Q218,Fântâneaua Rece River,Romania,5,278539,In what country is Fântâneaua Rece River?,"[""Romania"", ""Roumania"", ""Rumania"", ""România"", ..."
3331,Drăgăneasa River,country,3274434,Romania,1405201,182,756886,[],"[""Roumania"",""Rumania"",""Rom\u00e2nia"",""ro"",""\ud...",http://www.wikidata.org/entity/Q4716870,http://www.wikidata.org/entity/Q218,Drăgăneasa River (Provița),Romania,6,278539,In what country is Drăgăneasa River?,"[""Romania"", ""Roumania"", ""Rumania"", ""România"", ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,Kanye West,occupation,809091,record producer,345517,22,620958,"[""Kanye Omari West"",""Yeezy"",""Ye"",""Yeezus"",""The...","[""music producer""]",http://www.wikidata.org/entity/Q15935,http://www.wikidata.org/entity/Q183945,Kanye West,Record producer,940772,42844,What is Kanye West's occupation?,"[""record producer"", ""music producer"", ""songwri..."
11939,United Kingdom,capital,593981,London,242916,422,2840765,"[""\ud83c\uddec\ud83c\udde7"",""UK"",""United Kingd...","[""London, UK"",""London, United Kingdom"",""London...",http://www.wikidata.org/entity/Q145,http://www.wikidata.org/entity/Q84,United Kingdom,London,1066107,523018,What is the capital of United Kingdom?,"[""London"", ""London, UK"", ""London, United Kingd..."
9209,Elizabeth II,religion,6515620,Anglicanism,2925405,106,2145926,"[""Queen Elizabeth II"",""Elizabeth Alexandra Mar...","[""Anglicanism, Anglican Church""]",http://www.wikidata.org/entity/Q9682,http://www.wikidata.org/entity/Q6423963,Elizabeth II,Anglicanism,1291432,56744,What is the religion of Elizabeth II?,"[""Anglicanism"", ""Anglicanism, Anglican Church""]"
11761,India,capital,5015775,New Delhi,2215085,422,2938512,"[""Bharat"",""Hindustan"",""Bharatvarsh"",""in"",""IN"",...","[""New Delhi district"",""Nayi Dilli""]",http://www.wikidata.org/entity/Q668,http://www.wikidata.org/entity/Q987,India,New Delhi,1301086,128193,What is the capital of India?,"[""New Delhi"", ""New Delhi district"", ""Nayi Dilli""]"


In [21]:
condition_data = df.sort_values(by="s_pop", ascending=False).head(100).reset_index(drop=True)
condition_data

,subj,prop,id,obj,subj_id,prop_id,obj_id,s_aliases,o_aliases,s_uri,o_uri,s_wiki_title,o_wiki_title,s_pop,o_pop,question,possible_answers
0,United States of America,capital,2266274,"Washington, D.C.",988513,422,2024419,"[""the United States of America"",""America"",""U.S...","[""Washington"",""Washington DC"",""Washington, DC""...",http://www.wikidata.org/entity/Q30,http://www.wikidata.org/entity/Q61,United States,"Washington, D.C.",1629691,283465,What is the capital of United States of America?,"[""Washington, D.C."", ""Washington"", ""Washington..."
1,India,capital,5015775,New Delhi,2215085,422,2938512,"[""Bharat"",""Hindustan"",""Bharatvarsh"",""in"",""IN"",...","[""New Delhi district"",""Nayi Dilli""]",http://www.wikidata.org/entity/Q668,http://www.wikidata.org/entity/Q987,India,New Delhi,1301086,128193,What is the capital of India?,"[""New Delhi"", ""New Delhi district"", ""Nayi Dilli""]"
2,Elizabeth II,religion,6515620,Anglicanism,2925405,106,2145926,"[""Queen Elizabeth II"",""Elizabeth Alexandra Mar...","[""Anglicanism, Anglican Church""]",http://www.wikidata.org/entity/Q9682,http://www.wikidata.org/entity/Q6423963,Elizabeth II,Anglicanism,1291432,56744,What is the religion of Elizabeth II?,"[""Anglicanism"", ""Anglicanism, Anglican Church""]"
3,United Kingdom,capital,593981,London,242916,422,2840765,"[""\ud83c\uddec\ud83c\udde7"",""UK"",""United Kingd...","[""London, UK"",""London, United Kingdom"",""London...",http://www.wikidata.org/entity/Q145,http://www.wikidata.org/entity/Q84,United Kingdom,London,1066107,523018,What is the capital of United Kingdom?,"[""London"", ""London, UK"", ""London, United Kingd..."
4,Kanye West,occupation,809091,record producer,345517,22,620958,"[""Kanye Omari West"",""Yeezy"",""Ye"",""Yeezus"",""The...","[""music producer""]",http://www.wikidata.org/entity/Q15935,http://www.wikidata.org/entity/Q183945,Kanye West,Record producer,940772,42844,What is Kanye West's occupation?,"[""record producer"", ""music producer"", ""songwri..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,A.C. Milan,color,736578,red,313273,472,1036579,"[""AC Milan"",""Associazione Calcio Milan"",""Assoc...","[""red color""]",http://www.wikidata.org/entity/Q1543,http://www.wikidata.org/entity/Q3142,A.C. Milan,Red,188472,51772,What color is A.C. Milan?,"[""red"", ""red color""]"
96,Jurassic Park,screenwriter,1057843,David Koepp,453107,533,907664,[],[],http://www.wikidata.org/entity/Q167726,http://www.wikidata.org/entity/Q270178,Jurassic Park (film),David Koepp,184720,13666,Who was the screenwriter for Jurassic Park?,"[""Michael Crichton"", ""John Michael Crichton"", ..."
97,Lithuania,capital,2776008,Vilnius,1191903,422,752152,"[""Republic of Lithuania"",""Lietuva"",""\ud83c\udd...","[""Vilna"",""Wilno"",""Vilne"",""Wilna"",""Vi\u013c\u01...",http://www.wikidata.org/entity/Q37,http://www.wikidata.org/entity/Q216,Lithuania,Vilnius,183018,42978,What is the capital of Lithuania?,"[""Vilnius"", ""Vilna"", ""Wilno"", ""Vilne"", ""Wilna""..."
98,Wales,capital,1927140,Cardiff,847062,422,40007,"[""WAL"",""WLS"",""Cymru"",""Cambria""]","[""Caerdydd"",""Cardiff, Wales""]",http://www.wikidata.org/entity/Q25,http://www.wikidata.org/entity/Q10690,Wales,Cardiff,177730,50234,What is the capital of Wales?,"[""Cardiff"", ""Caerdydd"", ""Cardiff, Wales""]"


In [22]:
df["o_aliases"] = df["o_aliases"].apply(literal_eval)
df["possible_answers"] = df["possible_answers"].apply(literal_eval)
df["s_aliases"] = df["s_aliases"].apply(literal_eval)

In [23]:
facts = []
for i, row in df.iterrows():
    relation = row["prop"]
    possible_obj_ctxs = df[df["prop"] == relation]["obj"].unique()
    obj_ctx = row["obj"]
    obj_prior_possibilities = [row["obj"]] + row["o_aliases"]
    while obj_ctx in obj_prior_possibilities:
        obj_ctx = random.choice(possible_obj_ctxs)
        obj_ctx_aliases = df[df["obj"] == obj_ctx]["o_aliases"].iloc[0]
    extra_info_obj_ctxs_data = df[df["obj"] == obj_ctx]
    extra_info_obj_ctx_data = extra_info_obj_ctxs_data.sample(n=1)
    extra_info_obj_ctx = f"is the {extra_info_obj_ctx_data['prop'].item()} of {extra_info_obj_ctx_data['subj'].item()}"
    
    condition_row = condition_data.sample(n=1)
    condition = f"{condition_row['subj'].item()} is the {condition_row['prop'].item()} of {condition_row['obj'].item()}"
    counterfactual_condition = f"{condition_row['subj'].item()} were the {condition_row['prop'].item()} of {condition_row['obj'].item()}"
    facts.append(
        {
            "subject": row["subj"],
            "relation": row["prop"],
            "object_ctx": obj_ctx,
            "object_pri": row["obj"],
            "object_ctx_aliases": obj_ctx_aliases,
            "object_prior_aliases": row["o_aliases"],
            "possible_prior_answers": row["possible_answers"],
            "obj_prior_id": row["obj_id"],
            "subj_pop": row["s_pop"],
            "obj_pop": row["o_pop"],
            "subject_relation": f"the {row['prop']} of {row['subj']}",
            "extra_info_obj_ctx": extra_info_obj_ctx,
            "condition": condition,
            "counterfactual_condition": counterfactual_condition,
            "condition_index": int(condition_row.index[0])
        }
    )


In [24]:
facts

[{'subject': 'George Rankin',
  'relation': 'occupation',
  'object_ctx': 'biologist',
  'object_pri': 'politician',
  'object_ctx_aliases': ['biologists'],
  'object_prior_aliases': ['political leader',
   'political figure',
   'polit.',
   'pol'],
  'possible_prior_answers': ['politician',
   'political leader',
   'political figure',
   'polit.',
   'pol'],
  'obj_prior_id': 2834605,
  'subj_pop': 142,
  'obj_pop': 25692,
  'subject_relation': 'the occupation of George Rankin',
  'extra_info_obj_ctx': 'is the occupation of Jean-Baptiste de Lamarck',
  'condition': 'Halloween II is the director of Rob Zombie',
  'counterfactual_condition': 'Halloween II were the director of Rob Zombie',
  'condition_index': 83},
 {'subject': 'John Mayne',
  'relation': 'occupation',
  'object_ctx': 'lawyer',
  'object_pri': 'journalist',
  'object_ctx_aliases': ['attorney', 'solicitor', 'barrister', 'lawyers'],
  'object_prior_aliases': ['journo', 'journalists'],
  'possible_prior_answers': ['journa

In [25]:
from typing import List, Dict, Any
import jsonlines


def write_dicts_one_by_one(data: List[Dict[str, Any]], filename: str) -> None:
    """
    Alternative: Write dictionaries one by one using jsonlines.
    
    Args:
        data: List of dictionaries to write
        filename: Output filename (should end with .jsonl)
    """
    print(f"Writing {len(data)} records to {filename} (one by one)")
    failed_items = []
    with jsonlines.open(filename, mode='w') as writer:
        for item in data:
            try:
                writer.write(item)
            except Exception as e:
                failed_items.append(item)
                print(f"Error writing item: {e}")
                print(item)
    print(f"Failed to write {len(failed_items)} items")    
    print(f"Successfully wrote {len(data) - len(failed_items)}/{len(data)} records to {filename} (one by one)")

In [26]:
write_dicts_one_by_one(facts, "popqa_filtered.jsonl")

Writing 7195 records to popqa_filtered.jsonl (one by one)
Error writing item: 'utf-8' codec can't encode characters in position 164-167: surrogates not allowed
{'subject': 'Alastair Heathcote', 'relation': 'place of birth', 'object_ctx': 'Uganda', 'object_pri': 'Athens', 'object_ctx_aliases': ['Republic of Uganda', 'ug', '\ud83c\uddfa\ud83c\uddec', 'UGA'], 'object_prior_aliases': [], 'possible_prior_answers': ['Athens'], 'obj_prior_id': 299930, 'subj_pop': 161, 'obj_pop': 101288, 'subject_relation': 'the place of birth of Alastair Heathcote', 'extra_info_obj_ctx': 'is the place of birth of Louis Kasekende', 'condition': 'The Matrix is the director of The Wachowskis', 'counterfactual_condition': 'The Matrix were the director of The Wachowskis', 'condition_index': 52}
Error writing item: 'utf-8' codec can't encode characters in position 172-175: surrogates not allowed
{'subject': 'Austin Cuvillier', 'relation': 'place of birth', 'object_ctx': 'Egypt', 'object_pri': 'Quebec City', 'object